In [27]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2


In [28]:
df=pd.read_csv('train (1).csv')

In [29]:
df=df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [30]:
x_tr,x_te,y_tr,y_te=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2)

In [31]:
x_tr

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
764,3,male,16.0,0,0,7.7750,S
445,1,male,4.0,0,2,81.8583,S
783,3,male,NaN,1,2,23.4500,S
241,3,female,NaN,1,0,15.5000,Q
648,3,male,NaN,0,0,7.5500,S
...,...,...,...,...,...,...,...
689,1,female,15.0,0,1,211.3375,S
140,3,female,NaN,0,2,15.2458,C
785,3,male,25.0,0,0,7.2500,S
86,3,male,16.0,1,3,34.3750,S


In [32]:
y_tr

,Survived
764,0
445,1
783,0
241,1
648,0
...,...
689,1
140,0
785,0
86,0


# Imputation


In [33]:
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])

],remainder='passthrough')

# OneHotEnc

In [34]:
trf2=ColumnTransformer([
    ('ohe_sex_emb',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]),
    ], remainder='passthrough')

# Scaler
we use min max due to scaling that we are doin


In [35]:
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) # slice use to apply code on all
    ])


# feature seltion

In [36]:
trf4= SelectKBest(score_func=chi2,k=8)

In [37]:
trf5= DecisionTreeClassifier()

# **Create Pipeline**

In [38]:
pipe= Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
   ('trf4',trf4),
    ('trf5',trf5)
    ])

# Pipeline VS Make_Pipeline
naming step of pipe line is not in make_pipeline

In [39]:
#pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

# train

In [40]:
# for diagram
from sklearn import set_config
set_config(display='diagram')

In [41]:
pipe.fit(x_tr,y_tr) # if you r using algorith than use fit and if u r not using algo than use fit_transform

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emb',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7aaf0f01da80>)),
                ('trf5', DecisionTreeClassifier())])

# we can enter in any list and find any thing

In [42]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [43]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_ # as it is list so we went in impute age[0] inside this we went simple imputer[1]
# this is mean value

array([29.9365614])

In [44]:
# as it is list so we went in impute emb[1] inside this we went simple imputer[1]
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [45]:
y_pred= pipe.predict(x_te)

In [46]:
y_te

,Survived
361,0
402,0
649,1
257,1
623,0
...,...
184,1
576,1
391,1
76,0


In [47]:
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0])

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te,y_pred)

0.6480446927374302

In [49]:
# less due to feature selection

In [50]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))